# Face-Mask-Classification Project


Authors:
+ Tobias Palmowski
+ Fabian Metz
+ Thilo Sander

Date of Midterm-Report: 29.03.2021 <br>
Date of final submission: 26.04.2021


### Introduction

This Jupyter Notebook is the core of the Face-Mask-Classification Project performed in the class "Machine Learning" of the Hertie School in Berlin. There is one other Jupyter Notebook which deals with combining the different datasets into one large data set - a task only performed once and therefore outsourced to another file.


### Data Processing: Pipeline-Building

<br>
<br>
<br>
<br>
[Short Description]

In [160]:
# Import necessary libraries and set-up Jupyter Notebook.

# Common imports
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

# Imports for dealing with images:
import PIL #Pillow (install with "pip install Pillow")

# to make this notebook's output stable across runs (safety measure)
np.random.seed(42)

# Set path to correct and incorrect data sets for keeping references short later
ROOT_DATA = "01_data/99_dummy_toy_data"
PATH_DATA_CORRECT = os.path.join(ROOT_DATA + "/correct")
PATH_DATA_INCORRECT = os.path.join(ROOT_DATA + "/incorrect")

# Where to save possible figures
PROJECT_ROOT_DIR = "02_figures"
CHAPTER_ID = "01_data_preparation"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [161]:
# Open pickle file that contains the directory
import pickle
pic_data = pickle.load(open(os.path.join(ROOT_DATA + "/cleaned/pic_data.pkl"),"rb"))

In [162]:
# Exploring dictionary structure
pic_data

{'rgb_data': array([[ 48.,  28.,  18., ..., 141., 205., 255.],
        [140., 160., 177., ...,  89.,  58.,  56.],
        [212., 204., 189., ..., 196., 171., 131.],
        [ 48.,  28.,  18., ..., 141., 205., 255.],
        [140., 160., 177., ...,  89.,  58.,  56.],
        [212., 204., 189., ..., 196., 171., 131.]]),
 'labels': array([1, 1, 1, 0, 0, 0], dtype=uint8)}

In [163]:
# Copying dictionary data into separate data frames
rgb_data, labels = pic_data["rgb_data"], pic_data["labels"]

In [164]:
# Explore dimensionalities of data frames
print("Dimensions of rgb_data:", rgb_data.shape)
print("Dimensions of labels:", labels.shape)

Dimensions of rgb_data: (6, 12288)
Dimensions of labels: (6,)


In [165]:
# Split into test and training data set
from sklearn.model_selection import train_test_split

rgb_data_train, rgb_data_test, labels_train, labels_test = train_test_split(rgb_data, labels, test_size=0.33, random_state=42)

### Baseline: SGD Classifier

<br>

This section defines a Stochastic Gradient Decent method as a baseline for the project.
[Short Description]

In [166]:
# redefining labels as True False
labels_train_tf = (labels_train == 1)
labels_test_tf = (labels_test == 1)


# Code for Baseline
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42) 
sgd_clf.fit(rgb_data_train, labels_train_tf)

SGDClassifier(random_state=42)

Parameters need to be checked, what makes sense for a toy dataset?

### Evaluation of Baseline

<br>
The normal Evaluation does not work with the current toy dataset as its too small.

<div class="alert alert-block alert-danger">
<b>ATTENTION</b>
<p>
The Evaluation Part is coded well, but does not work with the small test toy dataset.
</div>

In [167]:
# Cross Valuation Score
from sklearn.model_selection import cross_val_score
cvs = cross_val_score(sgd_clf, rgb_data_train, labels_train_tf, cv=3, scoring="accuracy")

/Users/tobiaspalmowski/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"
/Users/tobiaspalmowski/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/tobiaspalmowski/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/tobiaspalmowski/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py", line 725, in fit
    return self._fit(X, y, alpha=self.alpha, C=1.0,
  File "/Users/tobiaspalmowski/opt/anaconda3/lib/python3.8/site-packages/sklearn/lin

<div class="alert alert-block alert-danger">
<b>Place to work on</b>
<p>
CV needs to be specified for whole dataset.
</div>

In [168]:
#create prediction sets
from sklearn.model_selection import cross_val_predict
labels_train_pred = cross_val_predict(sgd_clf, rgb_data_train, labels_train_tf, cv = 3)


#Calculate Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_train_tf, labels_train_pred)


/Users/tobiaspalmowski/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


ValueError: The number of classes has to be greater than one; got 1 class

<div class="alert alert-block alert-danger">
<b>Place to work on</b>
<p>
CV needs to be specified for whole dataset.
</div>

In [ ]:
# precision and recall
from sklearn.metrics import precision_score, recall_score
ps = precision_score(labels_train_tf, labels_train_pred)
rs = recall_score(labels_train_tf, labels_train_pred)

### Preliminary Output for Working Process
<br>
This section gives us the output for the confusion matrix, the cross validation score and the precision and recall stores for the current code and tuning.

In [ ]:
# Output for us while working on it
print("Confusion Matrix")
pd.DataFrame(cm)

In [ ]:
print('Cross Validation Scores')
print(cvs)

In [ ]:
print("Precision Score")
print(ps)

In [ ]:
print("Recall Score")
print(rs)

<div class="alert alert-block alert-danger">
<b>Place to work on</b>
<p>
What output do we want to generate?
</div>